In [88]:
import cv2
import numpy as np
import os
from google.colab import drive
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Load pre-trained feature extractor (VGG16)
def load_feature_extractor():
    model = VGG16(weights="imagenet", include_top=False, pooling="avg")
    return model

# Extract features from an image
def extract_features(image_path, model):
    image = load_img(image_path, target_size=(224, 224))
    image_array = img_to_array(image)
    image_array = np.expand_dims(image_array, axis=0)
    image_array = preprocess_input(image_array)
    features = model.predict(image_array)
    return features.flatten()

# Load the feature extractor
feature_extractor = load_feature_extractor()
print("Feature extractor loaded.")


Feature extractor loaded.


In [89]:
# Mount Google Drive
drive.mount('/content/drive')

# Build a dataset of features from a folder
def build_feature_dataset(folder_path, model):
    dataset_features = []
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if file_name.lower().endswith((".jpg", ".jpeg", ".png")):
            features = extract_features(file_path, model)
            dataset_features.append(features)
    return np.array(dataset_features)

# Path ke folder dataset
dataset_folder = "/content/drive/My Drive/dataset_pcd/eagle_dataset"

# Build the feature dataset
print("Building feature dataset...")
dataset_features = build_feature_dataset(dataset_folder, feature_extractor)
print("Feature dataset built.")

# Normalize and train One-Class SVM
print("Training One-Class SVM...")
scaler = StandardScaler()
scaled_features = scaler.fit_transform(dataset_features)
oc_svm = OneClassSVM(kernel="rbf", gamma="auto", nu=0.1)
oc_svm.fit(scaled_features)
print("One-Class SVM trained.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Building feature dataset...
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 629ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 501ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 482ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 800ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 487ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 498ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 486ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 495ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 483ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 519ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 484ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 486ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 481ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 485ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 493ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 555ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 859ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 855ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 851ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 781ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 905ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━

In [91]:
#EAGLE TEST
# Predict if images in a folder are in the same class as the dataset
def predict_images_in_folder(input_folder_path, model, oc_svm, scaler):
    results = []
    for file_name in os.listdir(input_folder_path):
        file_path = os.path.join(input_folder_path, file_name)
        if file_name.lower().endswith((".jpg", ".jpeg", ".png")):
            try:
                is_eagle, decision_score = predict_image(file_path, model, oc_svm, scaler)
                if is_eagle:
                    result = f"{file_name}: Eagle detected with decision score: {decision_score:.2f}"
                else:
                    result = f"{file_name}: Non-eagle detected in the input image"
                results.append(result)
            except Exception as e:
                results.append(f"{file_name}: Error processing image - {e}")
    return results

# Path ke folder input
input_folder = "/content/drive/My Drive/dataset_pcd/eagle"

# Analyze images in the folder
print("Analyzing all images in the folder...")
results = predict_images_in_folder(input_folder, feature_extractor, oc_svm, scaler)

# Print results
for result in results:
    print(result)


Analyzing all images in the folder...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 492ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 510ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 471ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 498ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 477ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 503ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 479ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 491ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 477ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 759ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 863ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 865ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 820ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 783ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 886ms/step
input_image_13.jpg: Eagle detected with decision score: 0.76
input_image_7.jpg: Eagle detected with decision score: 0.59
input_image_8.jpg: Eagle detected with decision score: 0.80
input_image_6.jpg: Eagle detected with decision score: 0.57
input_image_9.jpg: Eagle detected with decision score: 0.73
input_image_4.jpg: Eagle detected with decision score: 0.51
input_image_3.jp

In [92]:
# NON EAGLE TEST
# Predict if images in a folder are in the same class as the dataset
def predict_images_in_folder(input_folder_path, model, oc_svm, scaler):
    results = []
    for file_name in os.listdir(input_folder_path):
        file_path = os.path.join(input_folder_path, file_name)
        if file_name.lower().endswith((".jpg", ".jpeg", ".png")):
            try:
                is_eagle, decision_score = predict_image(file_path, model, oc_svm, scaler)
                if is_eagle:
                    result = f"{file_name}: Eagle detected with decision score: {decision_score:.2f}"
                else:
                    result = f"{file_name}: Non-eagle detected in the input image"
                results.append(result)
            except Exception as e:
                results.append(f"{file_name}: Error processing image - {e}")
    return results

# Path ke folder input
input_folder = "/content/drive/My Drive/dataset_pcd/non_eagle"

# Analyze images in the folder
print("Analyzing all images in the folder...")
results = predict_images_in_folder(input_folder, feature_extractor, oc_svm, scaler)

# Print results
for result in results:
    print(result)


Analyzing all images in the folder...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 495ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 477ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 491ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 475ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 492ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 492ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 501ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 498ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 475ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 489ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 469ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 485ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 473ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 488ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 471ms/step
input_image_1.jpg: Non-eagle detected in the input image
input_image_2.jpg: Non-eagle detected in the input image
input_image_3.jpg: Non-eagle detected in the input image
input_image_4.jpg: Non-eagle detected in the input image
input_image_5.jpg: Non-eagle detected in the input image
input_image_6.jpeg: Non-eagle detected in the input image
input_image_7.jpg: Non-eagle detec